In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import re

# 1- مسار ملف HTML
input_file = r"مكان الفايل الاساسي "

# 2- مسار ملف Excel النهائي
output_file = r"مكان استخراج الملف"

# 3- قراءة ملف HTML
with open(input_file, "r", encoding="utf-8") as f:
    soup = BeautifulSoup(f, "html.parser")

# 4- تجهيز القوائم
titles, channels, dates, times, datetimes = [], [], [], [], []

# 5- استخراج كل العناصر
events = soup.find_all("div", class_="content-cell")

# Regex علشان نمسك التاريخ
date_pattern = re.compile(r"[A-Za-z]{3}\s\d{1,2},\s\d{4},\s\d{1,2}:\d{2}:\d{2}\s[AP]M\s\w+")

for block in events:
    try:
        # العنوان
        video_tag = block.find_all("a")[0] if len(block.find_all("a")) > 0 else None
        title = video_tag.get_text(strip=True) if video_tag else "N/A"

        # القناة
        channel_tag = block.find_all("a")[1] if len(block.find_all("a")) > 1 else None
        channel = channel_tag.get_text(strip=True) if channel_tag else "N/A"

        # نجيب النصوص كلها بالترتيب
        texts = list(block.stripped_strings)

        # ندور على نص التاريخ
        date_part = None
        for t in texts:
            if date_pattern.search(t):
                date_part = date_pattern.search(t).group()
                break

        # نحول التاريخ
        dt = pd.to_datetime(date_part, errors="coerce") if date_part else None

        if pd.notna(dt):
            date = dt.strftime("%Y-%m-%d")
            time = dt.strftime("%I:%M %p")  # صيغة 12 ساعة AM/PM
            datetime_full = dt.strftime("%Y-%m-%d %I:%M %p")
        else:
            date, time, datetime_full = "N/A", "N/A", "N/A"

        # نجمع النتايج
        titles.append(title)
        channels.append(channel)
        dates.append(date)
        times.append(time)
        datetimes.append(datetime_full)

    except Exception as e:
        print("خطأ في معالجة:", e)

# 6- إنشاء DataFrame
df = pd.DataFrame({
    "Title": titles,
    "Channel": channels,
    "Date": dates,
    "Time": times,
    "Datetime": datetimes
})

# 7- إزالة التكرارات
df = df.drop_duplicates()

# 8- حفظ النتائج في Excel
df.to_excel(output_file, index=False)

print("✅ تم استخراج الملف بالكامل وحفظه في:", output_file)
print("إجمالي السجلات:", len(df))
